In [1]:
import os

In [2]:
os.chdir('C:/Users/mohit/Downloads/Compressed/archive_2')
dir = os.getcwd()
dir

'C:\\Users\\mohit\\Downloads\\Compressed\\archive_2'

In [3]:
train_data = os.path.join('train')
test_data = os.path.join('test')

In [7]:
files = os.listdir(train_data)
print(files[:3])

['angry', 'disgusted', 'fearful']


### CNN model for training 

In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3,3) , input_shape=(48,48,3) , activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512 , activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(7 , activation='softmax')
    
])

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 46, 46, 8)         224       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 21, 21, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 4, 32)         

In [33]:
model.compile(loss = 'CategoricalCrossentropy',
             optimizer=RMSprop(learning_rate=0.001),
             metrics=['accuracy'])

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale = 1/255)

train_data_gen = train_gen.flow_from_directory(train_data , 
                                              target_size = (48,48),
                                              batch_size=10,
                                              class_mode="categorical")

test_data_gen = train_gen.flow_from_directory(test_data ,
                                             target_size = (48,48),
                                             batch_size = 10,
                                             class_mode = "categorical")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [35]:
model_fiting = model.fit(train_data_gen , steps_per_epoch=8,epochs=10 ,validation_data=test_data_gen, verbose=1 ,batch_size =10)

Epoch 1/10
8/8 [==============================] - 23s 3s/step - loss: 2.0547 - accuracy: 0.1250 - val_loss: 1.8744 - val_accuracy: 0.1833
Epoch 2/10
8/8 [==============================] - 3s 353ms/step - loss: 1.9158 - accuracy: 0.1500 - val_loss: 1.8651 - val_accuracy: 0.1432
Epoch 3/10
8/8 [==============================] - 2s 334ms/step - loss: 1.8514 - accuracy: 0.2125 - val_loss: 1.8413 - val_accuracy: 0.1942
Epoch 4/10
8/8 [==============================] - 2s 340ms/step - loss: 1.8362 - accuracy: 0.2625 - val_loss: 1.8604 - val_accuracy: 0.1718
Epoch 5/10
8/8 [==============================] - 2s 339ms/step - loss: 1.8390 - accuracy: 0.1500 - val_loss: 1.8394 - val_accuracy: 0.1739
Epoch 6/10
8/8 [==============================] - 2s 339ms/step - loss: 1.9321 - accuracy: 0.1500 - val_loss: 1.8548 - val_accuracy: 0.2076
Epoch 7/10
8/8 [==============================] - 2s 324ms/step - loss: 1.8851 - accuracy: 0.2500 - val_loss: 1.8349 - val_accuracy: 0.2471
Epoch 8/10
8/8 [======